In [1]:
import pandas as pd
import sys
import os
import numpy as np
import yfinance as yf
import torch.nn as nn
from sklearn.model_selection import train_test_split

import sys
import os
sys.path.append(os.path.abspath("../src"))
from data_prep_utils import *

In [2]:
context_tickers = {
    '30_year_bonds': 'TLT',
    '10_year_bonds': 'IEF',
    '5_year_bonds': 'IEI',
    '2_year_bonds': 'SHY',
    '1_year_bonds': 'SHV',
    'gold': 'GLD',
    'silver': 'SLV',
    # 'copper': 'CPER', # starts in 2011
    'oil': 'USO',
    'natural_gas': 'UNG',
    'sp500': 'SPY',
    'nasdaq': 'QQQ',
    'dow_jones': 'DIA',
    'russell_2000': 'IWM',
    'us_dollar': 'UUP',
    'emerging_markets': 'EEM',
    'euro': 'FXE',
    'british_pound': 'FXB',
    'japanese_yen': 'FXY',
    # 'bitcoin': 'BTC-USD', # starts in 2014
    # 'ethereum': 'ETH-USD', # start in 2014
}

target_tickers = {
    'costco': 'COST',
    'coinbase': 'COIN',
    'robinhood': 'HOOD',
    'amazon': 'AMZN',
    'apple': 'AAPL',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'tesla': 'TSLA',
    'meta': 'META',
    'nvidia': 'NVDA',
    'general_motors': 'GM',
    'ford': 'F',
    'crowdstrike': 'CRWD',
    'palantir': 'PLTR',
}

START_DATE = '2010-01-01'
END_DATE = '2025-06-20'
# only 60 days of minute data available from yfinance
MINUTE_END_DATE = pd.to_datetime(END_DATE) - pd.Timedelta(days=60)


In [3]:
# Download and enrich data
context_data = download_and_enrich_data(context_tickers, period = 'max', interval='1d')    
context_df = pd.concat(context_data.values(), axis=1) # .dropna()
target_data = download_and_enrich_data(target_tickers, period = 'max', interval='1d')    
target_df = pd.concat(target_data.values(), axis=1)

# verify timeframes
# for ticker, df in context_data.items():
#     print(f"{ticker} index range: {df.index.min()} to {df.index.max()}", 'count:', len(df))

context_df.to_csv('../data/daily_context_data.csv')
target_df.to_csv('../data/daily_target_data.csv')

print('Num days in context_df:', len(context_df))
print('Num days in target_df:', len(target_df))
print(context_df.shape)
display(context_df.tail(2 ))

c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period = period, interval=interval, progress=False)
c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period = period, interval=interval, progress=False)
c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period = period, interval=interval, progress=False)
c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period = period, interval=interval, progress=False)
c:\Users\saris\OneDrive\Desktop\active inf investing

Num days in context_df: 8139
Num days in target_df: 13360
(8139, 234)


,TLT_Close,TLT_High,TLT_Low,TLT_Open,TLT_Volume,TLT_EMA_12,TLT_EMA_26,TLT_MACD,TLT_Signal,TLT_BB_upper,...,FXY_Open,FXY_Volume,FXY_EMA_12,FXY_EMA_26,FXY_MACD,FXY_Signal,FXY_BB_upper,FXY_BB_lower,FXY_OBV,FXY_ATR
Date,,,,,,,,,,,,,,,,,,,,,
2025-06-25,87.510002,87.510002,86.800003,87.110001,26345300.0,86.596734,86.349947,0.246787,0.024806,87.657386,...,63.150002,139100.0,63.529562,63.638991,-0.109428,-0.040914,64.567434,62.934566,34792500.0,0.455714
2025-06-26,87.949997,87.949997,87.200699,87.598999,30008320.0,86.804929,86.468469,0.336459,0.087137,87.911163,...,63.810001,252385.0,63.568091,63.649436,-0.081345,-0.049000,64.573161,62.944839,35044885.0,0.458107


In [6]:
context_data = download_and_enrich_data(context_tickers, period='60d', interval='1m')    
context_df = pd.concat(context_data.values(), axis=1).dropna()
target_data = download_and_enrich_data(target_tickers, period='60d', interval='1m')    
target_df = pd.concat(target_data.values(), axis=1)

context_df.to_csv('../data/minute_context_data.csv')
target_df.to_csv('../data/minute_target_data.csv')

print('Num mins in context_df:', len(context_df))
print('Num mins in target_df:', len(target_df))
print(context_df.shape)
display(context_df.tail(2))


c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period = period, interval=interval, progress=False)

1 Failed download:
['TLT']: YFPricesMissingError('possibly delisted; no price data found  (period=60d) (Yahoo error = "1m data not available for startTime=1745788649 and endTime=1750972649. Only 8 days worth of 1m granularity data are allowed to be fetched per request.")')
c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period = period, interval=interval, progress=False)

1 Failed download:
['IEF']: YFPricesMissingError('possibly delisted; no price data found  (period=60d) (Yahoo error = "1m data not available for startTime=1745788650 and endTime=1750972650. Only 8 days worth of 1m granularity data are allowed to be f

Num mins in context_df: 0
Num mins in target_df: 0
(0, 252)


,TLT_Adj Close,TLT_Close,TLT_High,TLT_Low,TLT_Open,TLT_Volume,TLT_EMA_12,TLT_EMA_26,TLT_MACD,TLT_Signal,...,FXY_Open,FXY_Volume,FXY_EMA_12,FXY_EMA_26,FXY_MACD,FXY_Signal,FXY_BB_upper,FXY_BB_lower,FXY_OBV,FXY_ATR
Date,,,,,,,,,,,,,,,,,,,,,


In [5]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset

# Define horizons for target predictions (trading days)
horizons = {"1_day": 1, "1_week": 5, "1_month": 21, "1_year": 252}

# Step 1: Create Target Columns
def create_targets(df, target_column, horizons):
    """
    Generate shifted target columns for specified horizons.
    """
    for horizon, shift in horizons.items():
        df[f"Target_{horizon}"] = df[target_column].shift(-shift)
    return df

# Step 2: Normalize Features
def normalize_features(df, exclude_columns):
    """
    Normalize input features using Min-Max scaling, excluding specified columns.
    """
    scaler = MinMaxScaler()
    input_features = df.drop(columns=exclude_columns)
    scaled_features = scaler.fit_transform(input_features)
    
    # Reconstruct normalized DataFrame
    normalized_df = pd.DataFrame(scaled_features, columns=input_features.columns, index=input_features.index)
    for column in exclude_columns:
        normalized_df[column] = df[column]  # Add back excluded columns (e.g., targets)
    
    return normalized_df, scaler

# Step 3: Create Sequences for Transformer Input
def create_sequences(data, target_columns, seq_length):
    """
    Create sequences for transformer input with specified sequence length.
    """
    X, y = [], []
    data_values = data.drop(columns=target_columns).values
    target_values = data[target_columns].values

    for i in range(len(data) - seq_length):
        X.append(data_values[i:i + seq_length])
        y.append(target_values[i + seq_length])
    
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Step 4: Custom Dataset for PyTorch
class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Full Data Preparation Pipeline
def prepare_data_pipeline(df, target_column, sequence_length=30):
    """
    Prepare data for transformer input: normalize, create targets, and generate sequences.
    """
    # Create targets
    df = create_targets(df, target_column, horizons)

    # Normalize features
    exclude_columns = [f"Target_{horizon}" for horizon in horizons.keys()]
    df, scaler = normalize_features(df, exclude_columns)

    # Drop NaN values
    df.dropna(inplace=True)

    # Split data into training, validation, and test sets
    train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)
    train_df, val_df = train_test_split(train_df, test_size=0.2, shuffle=False)

    # Create sequences
    target_columns = exclude_columns
    train_X, train_y = create_sequences(train_df, target_columns, sequence_length)
    val_X, val_y = create_sequences(val_df, target_columns, sequence_length)
    test_X, test_y = create_sequences(test_df, target_columns, sequence_length)

    # Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(StockDataset(train_X, train_y), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(StockDataset(val_X, val_y), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(StockDataset(test_X, test_y), batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, scaler

# Example Usage
# Assuming combined_df is the enriched DataFrame
target_column = target_tickers['costco'] + "_Close"
sequence_length = 256 # 21 = 1 month, 252 = 1 year

# Prepare the data
train_loader, val_loader, test_loader, scaler = prepare_data_pipeline(combined_df, target_column, sequence_length)

# Display the shapes of the datasets
print(f"Train Loader: {len(train_loader.dataset)} samples")     
print(f"Validation Loader: {len(val_loader.dataset)} samples")
print(f"Test Loader: {len(test_loader.dataset)} samples")


NameError: name 'combined_df' is not defined